In [2]:
%load_ext autoreload
%autoreload 2

import sys
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(sys.stderr)
formatter = logging.Formatter('%(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger.handlers = [handler]

import torch
from torch.autograd import Variable
import os
from metal.mmtl.cxr.analysis.notebooks.cxr_analysis import load_log_json, load_results_from_log
from metal.mmtl.cxr.cxr_tasks import create_tasks_and_payloads

In [3]:
# Setting env variables
os.environ['CXRDATA']='/data/datasets/mmtl/cxr'
os.environ['CXR8IMAGES']='/data/datasets/nih/images/images'

In [4]:
# Loading results from model log
model_log = '/home/jdunnmon/Research/repos/metal/logs/2019_05_07/test_naive_new_code_19_21_28'
log_jsons = load_results_from_log(model_log)

In [5]:
task_config = log_jsons['task_config']
task_config['subsample'] = -1
task_config['eval_finding'] = 'ALL'
task_names=['CXR8-DRAIN_ALL']
task_config['sample_dict']={"train_sample_dict":{'PNEUMOTHORAX':-1,'ALL':300}}
                            #"valid_sample_dict":{'PNEUMOTHORAX':-1,'ALL':300}}
task_config['add_normal_col']=True
task_config['slice_dict']={"CXR8-DRAIN_PNEUMOTHORAX": ["chest_drain_cnn_neg"]}
task_config['slice_pos_only']=['NONE']
task_config['active_slice_heads']={'pred':True,'ind':False}
task_config['verbose']=True
#task_config['train_sampler'] = "imbalanced_mmtl_sampler"
tasks, payloads = create_tasks_and_payloads(task_names, **task_config, batch_size=16)

metal.utils - INFO - Could not find kwarg "shuffle" in destination dict.
metal.utils - INFO - Could not find kwarg "attention" in destination dict.
metal.utils - INFO - Could not find kwarg "verbose" in destination dict.
metal.mmtl.cxr.cxr_tasks - INFO - Loading CXR8-DRAIN Dataset
metal.mmtl.cxr.cxr_tasks - INFO - Using train finding ALL
/home/jdunnmon/Research/repos/anaconda3/envs/metal/lib/python3.6/site-packages/torchvision/transforms/transforms.py:208: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")
metal.mmtl.cxr.cxr_datasets - INFO - Adding normal column to dataset
metal.mmtl.cxr.cxr_datasets - INFO - Using pre-specified sampling dictionary
metal.mmtl.cxr.cxr_datasets - INFO - Adding normal column to dataset
metal.mmtl.cxr.cxr_datasets - INFO - Adding normal column to dataset
metal.mmtl.cxr.cxr_tasks - INFO - Shuffling training split...
metal.mmtl.payload - INFO - Added label_se

In [6]:
# Checking to make sure our slice prediction task is reasonably balanced
from collections import Counter
print(Counter(payloads[1].data_loader.dataset.labels['CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred']))

Counter({2: 10715, 0: 336, 1: 208})


In [7]:
# Getting slice prediction task only
slice_prediction_task_name = 'CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred'
task_names = [t.name for t in tasks]
slice_prediction_task = tasks[task_names.index(slice_prediction_task_name)]
for p in payloads:
    p.labels_to_tasks = {k:v for k,v in p.labels_to_tasks.items() if slice_prediction_task_name in k}

In [8]:
from metal.mmtl.metal_model import MetalModel
# Initializing the model
model_config = log_jsons['model_config'] 
model_config['model_weights'] = None
model = MetalModel(tasks=[slice_prediction_task], **model_config)
missing, unexpected = model.load_weights(os.path.join('/home/jdunnmon/Research/repos/metal/logs/2019_05_07/test_naive_new_code_19_21_28','best_model.pth'))

metal.mmtl.metal_model - INFO - Your destination state dict has different keys for the update key.
metal.mmtl.metal_model - INFO - Loading without strictness.


In [ ]:
missing

In [23]:
# Training on dev!
import copy
train_on_dev = False
if train_on_dev:
    payloads[0] = copy.deepcopy(payloads[1])
    payloads[0].name = 'CXR8-DRAIN_train'
    payloads[0].split = 'train'

In [27]:
# Setting up trainer
from metal.mmtl.trainer import MultitaskTrainer
trainer_config = log_jsons['config'] 

tasks_to_freeze = [t.name for t in tasks if t.name != slice_prediction_task]
trainer_config["train_schedule_plan"] = {
                "plan": {
                    "-1": tasks_to_freeze,
                    },
                "freeze": "heads",
                }
trainer_config['n_epochs']=20
trainer_config['checkpoint_config']['checkpoint_metric'] = 'CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred/CXR8-DRAIN_valid/CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred/roc-auc'
trainer_config['checkpoint_config']['checkpoint_dir']='checkpoints_temp'
trainer_config['checkpoint_config']['checkpoint_metric_mode'] = 'max'
trainer_config['writer_config']['run_name'] = 'test_slice_train_exp'

# Initializing trainer 
trainer = MultitaskTrainer(**trainer_config)

In [28]:
# Training model
trainer.train_model(model, payloads, train_schedule_plan=trainer_config['train_schedule_plan'])

/home/jdunnmon/Research/repos/metal/metal/mmtl/trainer.py:643: UserWarning: You have provided checkpoint_dir, overriding the default of using log_dir/run_dir/run_name/checkpoints. Be careful: multiple concurrent runs may override each other.
  warnings.warn(msg)


[1.0 epo]: CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred:[CXR8-DRAIN_train/CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred/loss=6.70e-01, CXR8-DRAIN_valid/CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred/f1=4.89e-02, CXR8-DRAIN_valid/CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred/roc-auc=6.48e-01, CXR8-DRAIN_valid/CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred/accuracy=4.48e-01] model:[train/all/loss=6.70e-01, train/all/lr=9.50e-05, valid/all/loss=8.74e-01]
Saving model at iteration 1.00 with best (max) score CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred/CXR8-DRAIN_valid/CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred/roc-auc=0.648
[2.0 epo]: CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred:[CXR8-DRAIN_train/CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred/loss=5.88e-01, CXR8-DRAIN_valid/CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred/f1=5.29e-02, CXR8-DRAIN_valid/CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:

/home/jdunnmon/Research/repos/metal/metal/mmtl/trainer.py:1007: UserWarning: MeTaL does not support calculating loss on the test set.
  warnings.warn(msg)


{'CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred/CXR8-DRAIN_test/CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred/accuracy': 0.7848459539717891,
 'CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred/CXR8-DRAIN_test/CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred/f1': 0.04803941695750359,
 'CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred/CXR8-DRAIN_test/CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred/roc-auc': 0.7182708976040826,
 'CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred/CXR8-DRAIN_train/CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred/accuracy': 0.8318232044198896,
 'CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred/CXR8-DRAIN_train/CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred/f1': 0.7772900204858062,
 'CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred/CXR8-DRAIN_train/CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred/roc-auc': 0.9038623348493986,
 'CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred

{'CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred/CXR8-DRAIN_test/CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred/f1': 0.04803941695750359,
 'CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred/CXR8-DRAIN_test/CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred/roc-auc': 0.7182708976040826,
 'CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred/CXR8-DRAIN_test/CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred/accuracy': 0.7848459539717891,
 'model/valid/all/loss': 0.5271585620492956,
 'CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred/CXR8-DRAIN_valid/CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred/f1': 0.08019246190858059,
 'CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred/CXR8-DRAIN_valid/CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred/roc-auc': 0.6811988495638752,
 'CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred/CXR8-DRAIN_valid/CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred/accuracy': 0.7899844365101163,
 'CXR8-DRA

## SANDBOX

In [272]:
payloads[0].labels_to_tasks

{'CXR8-DRAIN_ATELECTASIS': 'CXR8-DRAIN_ATELECTASIS',
 'CXR8-DRAIN_CARDIOMEGALY': 'CXR8-DRAIN_CARDIOMEGALY',
 'CXR8-DRAIN_EFFUSION': 'CXR8-DRAIN_EFFUSION',
 'CXR8-DRAIN_INFILTRATION': 'CXR8-DRAIN_INFILTRATION',
 'CXR8-DRAIN_MASS': 'CXR8-DRAIN_MASS',
 'CXR8-DRAIN_NODULE': 'CXR8-DRAIN_NODULE',
 'CXR8-DRAIN_PNEUMONIA': 'CXR8-DRAIN_PNEUMONIA',
 'CXR8-DRAIN_PNEUMOTHORAX': 'CXR8-DRAIN_PNEUMOTHORAX',
 'CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred': 'CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred',
 'CXR8-DRAIN_CONSOLIDATION': 'CXR8-DRAIN_CONSOLIDATION',
 'CXR8-DRAIN_EDEMA': 'CXR8-DRAIN_EDEMA',
 'CXR8-DRAIN_EMPHYSEMA': 'CXR8-DRAIN_EMPHYSEMA',
 'CXR8-DRAIN_FIBROSIS': 'CXR8-DRAIN_FIBROSIS',
 'CXR8-DRAIN_PLEURAL_THICKENING': 'CXR8-DRAIN_PLEURAL_THICKENING',
 'CXR8-DRAIN_HERNIA': 'CXR8-DRAIN_HERNIA'}

In [243]:
slice_prediction_task

ClassificationTask(name=CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred, loss_multiplier=1.00)

In [278]:
import torch
#np.unique(payloads[0].data_loader.sampler.weights,return_counts=True)

AttributeError: 'SequentialSampler' object has no attribute 'weights'

In [276]:
payloads[1].data_loader.dataset.df.sum()

FOLLOW-UP #                                                       11370
PATIENT ID                                                     12032637
PATIENT AGE           038Y042Y038Y058Y041Y065Y065Y019Y054Y057Y048Y05...
PATIENT GENDER        FMMMMMMFMMMMFMMMFMMMFFMMMMFFMFMMFMMMFFFFFMFMMF...
VIEW POSITION         PAAPPAAPAPAPAPPAAPAPAPPAAPPAPAPAPAAPAPPAAPPAAP...
CARDIOMEGALY                                                         35
EMPHYSEMA                                                            95
EFFUSION                                                            180
HERNIA                                                               20
INFILTRATION                                                        169
MASS                                                                 87
NODULE                                                               80
ATELECTASIS                                                         130
PNEUMOTHORAX                                                    

In [280]:
import numpy as np
np.sum(payloads[1].data_loader.dataset.labels['CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred']==2)

262

In [181]:
import numpy as np
np.sum(payloads[1].data_loader.dataset.labels['CXR8-DRAIN_PNEUMOTHORAX']==1)

544

In [203]:
payloads[1].data_loader.dataset.labels

{'CXR8-DRAIN_ATELECTASIS': array([1, 1, 1, ..., 2, 2, 2]),
 'CXR8-DRAIN_CARDIOMEGALY': array([2, 2, 2, ..., 2, 2, 2]),
 'CXR8-DRAIN_EFFUSION': array([2, 2, 2, ..., 2, 2, 2]),
 'CXR8-DRAIN_INFILTRATION': array([1, 2, 2, ..., 2, 2, 2]),
 'CXR8-DRAIN_MASS': array([2, 2, 2, ..., 2, 2, 2]),
 'CXR8-DRAIN_NODULE': array([2, 2, 2, ..., 2, 2, 2]),
 'CXR8-DRAIN_PNEUMONIA': array([2, 2, 2, ..., 2, 2, 2]),
 'CXR8-DRAIN_PNEUMOTHORAX': array([2, 2, 2, ..., 2, 2, 2]),
 'CXR8-DRAIN_CONSOLIDATION': array([2, 2, 2, ..., 2, 2, 2]),
 'CXR8-DRAIN_EDEMA': array([2, 2, 2, ..., 2, 2, 2]),
 'CXR8-DRAIN_EMPHYSEMA': array([2, 2, 2, ..., 2, 2, 2]),
 'CXR8-DRAIN_FIBROSIS': array([2, 2, 2, ..., 2, 2, 2]),
 'CXR8-DRAIN_PLEURAL_THICKENING': array([2, 2, 2, ..., 2, 2, 2]),
 'CXR8-DRAIN_HERNIA': array([2, 2, 2, ..., 2, 2, 2]),
 'CXR8-DRAIN_NORMAL': array([2, 2, 2, ..., 1, 1, 1]),
 'CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred': array([0, 0, 0, ..., 0, 0, 0])}

In [238]:
tasks

[ClassificationTask(name=CXR8-DRAIN_ATELECTASIS, loss_multiplier=1.00),
 ClassificationTask(name=CXR8-DRAIN_CARDIOMEGALY, loss_multiplier=1.00),
 ClassificationTask(name=CXR8-DRAIN_EFFUSION, loss_multiplier=1.00),
 ClassificationTask(name=CXR8-DRAIN_INFILTRATION, loss_multiplier=1.00),
 ClassificationTask(name=CXR8-DRAIN_MASS, loss_multiplier=1.00),
 ClassificationTask(name=CXR8-DRAIN_NODULE, loss_multiplier=1.00),
 ClassificationTask(name=CXR8-DRAIN_PNEUMONIA, loss_multiplier=1.00),
 ClassificationTask(name=CXR8-DRAIN_PNEUMOTHORAX, loss_multiplier=1.00),
 ClassificationTask(name=CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred, loss_multiplier=1.00),
 ClassificationTask(name=CXR8-DRAIN_CONSOLIDATION, loss_multiplier=1.00),
 ClassificationTask(name=CXR8-DRAIN_EDEMA, loss_multiplier=1.00),
 ClassificationTask(name=CXR8-DRAIN_EMPHYSEMA, loss_multiplier=1.00),
 ClassificationTask(name=CXR8-DRAIN_FIBROSIS, loss_multiplier=1.00),
 ClassificationTask(name=CXR8-DRAIN_PLEURAL_THICKENING,